#

This notebook demonstrates how to connect to the Style Guard WebSocket service and send text for style checking.

In [17]:
import json
import websockets
import asyncio
from IPython.display import display, Markdown

## Define WebSocket Connection Function

In [19]:
async def connect_to_style_guard(text_samples):
    uri = "ws://localhost:8000/api/logic-inspector"

    character_summaries = """
Chapter 1:
Eva Rostova

Character Description
Eva is the captain of the Odyssey, an engineer and pilot faced with resource scarcity on Station Epsilon. She isn't a natural explorer but volunteered to pilot the Odyssey toward Xylos, driven by a belief in the mission's necessity and a personal connection to her daughter, Anya. She is motivated by the hope of discovering a new energy source and the well-being of Anya and the children on Epsilon. Eva carries a stone given to her by Anya, symbolizing her connection and motivation.

Character Relationships
Anya: Eva’s daughter, who inspires and motivates her through the gift of a stone and the hope of finding new energy.
Jax: Co-pilot on the mission to Xylos and a reluctant xenobotanist.
Jax

Character Description
Jax is Eva's co-pilot and the mission's reluctant xenobotanist, responsible for life support settings. Initially, he argued against the mission due to its risks but ultimately participated.

Character Relationships
Eva: Pilot and teammate on the mission to Xylos.
Anya

Character Description
Anya is Eva’s daughter, who has a supportive and influential role in Eva’s life. She gave Eva a "small, smooth stone," representing a source of inspiration or support during the mission.

Character Relationships
Eva: Her mother, whom she supports by giving a meaningful stone.

Chapter 2:
Eva

Character Description
Eva is portrayed as efficient and skilled in handling emergencies, particularly adept at identifying faults in the ship’s systems. She is decisive and urgent, especially when her daughter, Anya, is involved, prioritizing immediate solutions over standard protocols. Her dedication to the mission is driven by a personal obligation to her daughter's future, viewing their lives and the ship as tools to achieve it.

Character Relationships
Jax: Collaborates closely with Jax, convincing him to bypass standard procedures due to urgency and sharing the importance of their mission with him.
Jax

Character Description
Jax is a pragmatist who is quick to respond to emergencies through established procedures but shows initial hesitation to bypass them. He displays doubt and concern for risks but is also awed by their mission's significance.

Character Relationships
Eva: Works alongside Eva, relying on her expertise and respecting her decisiveness. He is involved in reaching the Xylos system with her, responding to her dedication.


    """
    location_summaries = """
    Xylos
    Chapter: 1
    Xylos is a planet with a red sun.
    """
    #
    async with websockets.connect(uri) as websocket:
        # Send initial style prompt
        await websocket.send(
            json.dumps(
                {
                    "character_summaries": character_summaries,
                    "location_summaries": location_summaries,
                }
            )
        )

        # Process each text sample
        for i, text in enumerate(text_samples):
            print(f"\n--- Sending Text Sample {i+1} ---")
            print(f"Text: {text[:100]}..." if len(text) > 100 else f"Text: {text}")

            # Send the text for checking
            await websocket.send(json.dumps({"text": text}))

            # Continue receiving messages until we get the "done" signal
            while True:
                response = await websocket.recv()
                response_data = json.loads(response)

                if response_data.get("status") == "done":
                    break

                print("\nResponse Received:")
                if "text_with_comments" in response_data:
                    display(
                        Markdown(
                            f"**Text with Comments:**\n{response_data['text_with_comments']}"
                        )
                    )
                else:
                    print(response_data)

            # Small delay between requests
            await asyncio.sleep(1)

## Define Sample Text and Style Guide

In [20]:
# Define some text samples for style checking
text_samples = [
    # "The product was developed by our team over a period of six months and it has been lauded by critics for its innovative features, user-friendly interface, and cutting-edge technology.",
    # "I believe that the implementation of the new system will significantly improve productivity, efficiency and customer satisfaction.",
    """Chapter 3: The Spark and the Flutter
The Xylos system was breathtaking. A binary pair of suns cast an ethereal, shifting light on the system's planets, painting them in hues of violet and gold. The third planet, designated Xylos-3, was a vibrant, oxygen-rich world, teeming with life unlike anything cataloged. And the energy signatures – they were stronger here, centered around a vast, crystalline forest that pulsed with a soft, internal light.
"Cap'n, look at this," Jax breathed, leaning over the scanner. "The isotope readings... they're off the charts. This isn't just an energy source, Eva. It's colossal. It could power a thousand colonies, maybe more, for centuries."
Eva felt a lump form in her throat. After all they had endured, all the doubt and hardship, it was real. The light. The salvation for Epsilon. Tears welled in her eyes, blurring the images on the screen – the pulsing forest, the thriving world. She thought of Anya, of the dim lights on the station, of the hope she was about to deliver.
"Prepare the landing sequence, Jax," she ordered, her voice thick with emotion. "We need to get ground samples, confirm viability, and plot extraction points. This is it."
As they guided the Odyssey towards a clearing near the crystalline forest, the ship's external cameras picked up movement below. Strange, six-legged creatures grazed on luminous moss. Exotic flora bloomed in explosions of color. It was a paradise.
Then, the camera feed zoomed in on something small, fluttering near the edge of the forest. It was... iridescent, with large, deep blue eyes and gossamer wings. It looked impossibly delicate, like something spun from pure light and air.
Eva froze. Her eyes widened, fixed on the creature. It wasn't the energy readings she was looking at anymore. It wasn't the pulsing forest, or the dying colony, or Anya's face in her mind. It was this tiny, ethereal being.
An inexplicable change washed over her. The crushing weight she had carried for months, the burning urgency, the unwavering focus on the mission – it didn't just lessen, it vanished. Replaced by a sudden, overwhelming fascination, a childish wonder.
"Eva? Cap'n? Everything alright?" Jax asked, noticing her stillness. "Readings are stable for landing."
Eva didn't respond immediately. She reached out, as if to touch the image on the screen. "It's... it's a Lumina Moth," she whispered, her voice soft, entirely different from the decisive commander of moments before. "I saw pictures in an old xenology text when I was a child. They were thought to be mythical."
"A... moth?" Jax frowned, confused. "Cap'n, the energy source is right there. We need to land, secure samples. Epsilon is waiting."
Eva turned away from the console, her eyes distant, sparkling with a strange, new light that had nothing to do with the life-saving energy pulsing below. "Landing can wait, Jax. Epsilon... Epsilon has survived this long. A little longer won't hurt."
Jax stared at her, dumbfounded. "A little longer? Eva, people are dying! We found it! The solution! Everything we worked for, everything you risked... for a... a moth?"
Eva walked past him, heading towards the ship's small observation port, her gaze fixed on the planet below, searching for the tiny, fluttering creature. "It's not just a moth, Jax. It's... it's beautiful. Unique. A marvel of life. We have to study it. Protect it. I need to understand it." She spoke with a quiet intensity, but it was an intensity directed entirely at this new, trivial interest, not the life-or-death mission they were on.
She turned back to Jax, her expression calm, almost serene, completely detached from the dire reality of their situation. "Cancel the landing near the energy source. Plot a course for that valley," she ordered, pointing vaguely towards an area away from the glowing forest. "I need to see that Lumina Moth up close. The energy can wait. My priority has changed."
Jax could only gape at her, the console lights reflecting the utter disbelief in his eyes. The fate of thousands, the desperate hope of a dying world, suddenly rendered secondary to the fleeting beauty of a space insect. Eva Rostova, the woman who would tear through the void for her daughter, had inexplicably abandoned everything at the finish line.
"""    
]

## Run the WebSocket Connection

In [21]:
# Run the websocket connection
try:
    await connect_to_style_guard(text_samples)
except Exception as e:
    print(f"Error connecting to websocket: {e}")


--- Sending Text Sample 1 ---
Text: Chapter 3: The Spark and the Flutter
The Xylos system was breathtaking. A binary pair of suns cast a...

Response Received:
{'status': 'logic_inspector_initialized'}

Response Received:
{'original_text': "The Xylos system was breathtaking. A binary pair of suns cast an ethereal, shifting light on the system's planets, painting them in hues of violet and gold.", 'comment': 'Inconsistency: Chapter 1 describes Xylos as having a single red sun, not a binary pair of suns.'}

Response Received:
{'original_text': 'Eva walked past him, heading towards the ship\'s small observation port, her gaze fixed on the planet below, searching for the tiny, fluttering creature. "It\'s not just a moth, Jax. It\'s... it\'s beautiful. Unique. A marvel of life. We have to study it. Protect it. I need to understand it."', 'comment': "Logical inconsistency: Eva's character has been motivated by the mission and her daughter's future. Her sudden shift to abandon the mission fo

## Alternative Approach with Manual Await (if needed)

In [ ]:
# If the await above doesn't work directly in Jupyter, use this approach
# asyncio.run(connect_to_style_guard(style_guide, text_samples))